In [1]:
import numpy as np
import pandas as pd
import torchvision
import os 
import sys

The first thing we realize is, that the Shastri et al dataset does not contain any information about the patients themselves and literally just the X-Rays. With the Cohen and all dataset we have more luck and do have metadata about the patients as well.

In [2]:
meta_cohen=pd.read_csv("data/Data_cohen_etal/metadata.csv")
meta_cohen

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,479,0.0,F,40.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,NaN,United Kingdom,images,072ecaf8c60a81980abb57150a8016_jumbo-9.jpeg,NaN,https://radiopaedia.org/cases/multifocal-round...,CC BY-NC-SA,Asthmatic. Shortness of breath and wheeze. Rou...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN
946,479,70.0,F,40.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,NaN,United Kingdom,images,ff33c406392b968d483174c97eb857_jumbo-9.jpeg,NaN,https://radiopaedia.org/cases/multifocal-round...,CC BY-NC-SA,Asthmatic. Shortness of breath and wheeze. The...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN
947,480,NaN,M,26.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,images,000001-266.jpg,NaN,https://www.eurorad.org/case/947,CC BY-NC-SA 4.0,fire-eater accidentally ingested a paraffin mi...,NaN,NaN
948,481,NaN,M,50.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,images,000001-272.jpg,NaN,https://www.eurorad.org/case/934,CC BY-NC-SA 4.0,"The patient, a heavy smoker, was referred to t...",NaN,NaN


We see that multiple images might come from the same patient. We pay attention to that in our sample description

In [64]:
exploration=meta_cohen.groupby("patientid").agg({
    "patientid": 'size',
    'offset':["mean", "std"],
    "sex": "first",
    "age": "first",
    "finding": "first", 
    "modality": (lambda x: str(x.unique()))
})
exploration[(      'sex',  'first')]=exploration[(      'sex',  'first')].apply(lambda x: "unknown" if x is None else x)
exploration

patientid     offset                sex   age  \
               size       mean       std    first first   
patientid                                                 
10                3  27.000000  8.000000        F  73.0   
100               1        NaN       NaN        F  62.0   
101               3        NaN       NaN        F  40.0   
102               1        NaN       NaN  unknown   NaN   
103               1        NaN       NaN  unknown   NaN   
...             ...        ...       ...      ...   ...   
95                3   1.000000  1.732051        F  70.0   
96                3   1.333333  1.527525        M  60.0   
97                4   1.500000  1.290994        F  70.0   
98                2        NaN       NaN        M  75.0   
99                1   0.000000       NaN        M  55.0   

                                  finding   modality  
                                    first   <lambda>  
patientid                                             
10                   Pneumonia/Viral/SARS  ['X-ray']  
100        Pneumonia/Bacterial/Klebsiella  ['X-ray']  
101         Pneumonia/Fungal/Pneumocystis  ['X-ray']  
102         Pneumonia/Fungal/Pneumocystis  ['X-ray']  
103         Pneumonia/Fungal/Pneumocystis  ['X-ray']  
...                                   ...        ...  
95               Pneumonia/Viral/COVID-19  ['X-ray']  
96               Pneumonia/Viral/COVID-19  ['X-ray']  
97               Pneumonia/Viral/COVID-19  ['X-ray']  
98               Pneumonia/Viral/COVID-19  ['X-ray']  
99               Pneumonia/Viral/COVID-19  ['X-ray']  

[472 rows x 7 columns]

In [65]:
findings=exploration.finding.value_counts()
sex=exploration.sex.value_counts()
modality=exploration.modality.value_counts()
age=exploration.agg({
    ("age","first"):["mean", "std"]
})

In [74]:
print(findings.to_latex())

\begin{tabular}{lr}
\toprule
 & count \\
first &  \\
\midrule
Pneumonia/Viral/COVID-19 & 307 \\
Pneumonia & 39 \\
todo & 25 \\
Pneumonia/Fungal/Pneumocystis & 17 \\
No Finding & 17 \\
Tuberculosis & 10 \\
Pneumonia/Bacterial/Streptococcus & 9 \\
Pneumonia/Viral/SARS & 8 \\
Pneumonia/Bacterial/Legionella & 6 \\
Pneumonia/Lipoid & 6 \\
Pneumonia/Bacterial/Klebsiella & 5 \\
Pneumonia/Viral/MERS-CoV & 4 \\
Pneumonia/Bacterial/Mycoplasma & 3 \\
Pneumonia/Viral/Influenza & 3 \\
Pneumonia/Fungal/Aspergillosis & 2 \\
Pneumonia/Viral/Varicella & 2 \\
Pneumonia/Viral/Influenza/H1N1 & 1 \\
Unknown & 1 \\
Pneumonia/Bacterial/E.Coli & 1 \\
Pneumonia/Viral/Herpes  & 1 \\
Pneumonia/Bacterial & 1 \\
Pneumonia/Bacterial/Chlamydophila & 1 \\
Pneumonia/Bacterial/Staphylococcus/MRSA & 1 \\
Pneumonia/Bacterial/Nocardia & 1 \\
Pneumonia/Aspiration & 1 \\
\bottomrule
\end{tabular}



In [76]:
sex

first  
M          270
F          158
unknown     44
Name: count, dtype: int64

In [78]:
age

,age
,first
mean,54.392157
std,17.213881


In [70]:
modality

<lambda>      
['X-ray']         427
['CT']             23
['X-ray' 'CT']     22
Name: count, dtype: int64

In [84]:
con=pd.concat([sex.reset_index(), age.reset_index(), modality.reset_index()], ignore_index=False, axis=1 )
print(con.to_latex())

\begin{tabular}{llrlrlr}
\toprule
 & first & count & ('index', '') & ('age', 'first') & <lambda> & count \\
\midrule
0 & M & 270 & mean & 54.392157 & ['X-ray'] & 427 \\
1 & F & 158 & std & 17.213881 & ['CT'] & 23 \\
2 & unknown & 44 & NaN & NaN & ['X-ray' 'CT'] & 22 \\
\bottomrule
\end{tabular}



# Images

In [88]:
from PIL import Image

file="data/Data_cohen_etal/train/NORMAL/NORMAL(0).jpg"


def get_resolution(file): 
    with Image.open(file) as img:
        width, height = img.size

    return width, height
get_resolution(file)

(1855, 1317)

In [97]:
def find_image_files(startpath):
    image_files = []
    img_file_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.tiff', '.bmp', '.webp']
    for foldername, subfolders, filenames in os.walk(startpath):
        for filename in filenames:
            if any(filename.lower().endswith(ext) for ext in img_file_extensions):
                relative_path = os.path.relpath(os.path.join(foldername, filename), startpath)
                relative_path=os.path.join(startpath,relative_path)
                image_files.append(relative_path)
    
    return image_files


def get_all_resolutions(path):
    return_dict={}
    image_files=find_image_files(path)
    for image_file in image_files:
        return_dict[image_file]=get_resolution(image_file)
    
    return return_dict


In [99]:
res_dict=get_all_resolutions("data/Data_cohen_etal/")
res_dict

{'data/Data_cohen_etal/train/COVID19/COVID19(48).jpg': (541, 557),
 'data/Data_cohen_etal/train/COVID19/COVID19(309).jpg': (1022, 1024),
 'data/Data_cohen_etal/train/COVID19/COVID19(173).jpg': (1384, 1326),
 'data/Data_cohen_etal/train/COVID19/COVID19(0).jpg': (2000, 2000),
 'data/Data_cohen_etal/train/COVID19/COVID19(148).jpg': (685, 754),
 'data/Data_cohen_etal/train/COVID19/COVID19(417).jpg': (2680, 2276),
 'data/Data_cohen_etal/train/COVID19/COVID19(260).jpg': (2000, 2000),
 'data/Data_cohen_etal/train/COVID19/COVID19(200).jpg': (2000, 2000),
 'data/Data_cohen_etal/train/COVID19/COVID19(378).jpg': (2567, 2190),
 'data/Data_cohen_etal/train/COVID19/COVID19(93).jpg': (1974, 1646),
 'data/Data_cohen_etal/train/COVID19/COVID19(448).jpg': (2199, 1926),
 'data/Data_cohen_etal/train/COVID19/COVID19(358).jpg': (1024, 1024),
 'data/Data_cohen_etal/train/COVID19/COVID19(314).jpg': (1024, 1024),
 'data/Data_cohen_etal/train/COVID19/COVID19(453).jpg': (2498, 1970),
 'data/Data_cohen_etal/train

In [7]:
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize(256),  # Resize the short side of the image to 256
    transforms.CenterCrop(224),  # Crop a square of 224x224 pixels from the center of the image
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image with ImageNet's mean and std
])


In [8]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


data_dir = "data/Data_cohen_etal"

# Load datasets
train_dataset = ImageFolder(data_dir + '/train', transform=transform)
test_dataset = ImageFolder(data_dir + '/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
!ls -l

total 193
drwxrwxr-x 6 leckhoff leckhoff  32768 Okt 11 06:26 data
-rw-rw-r-- 1 leckhoff leckhoff 125317 Okt 11 15:45 first_check.ipynb
drwxrwxr-x 7 leckhoff leckhoff  32768 Okt 11 12:52 lightning_logs
-rw-rw-r-- 1 leckhoff leckhoff     14 Okt 10 17:08 README.md


In [10]:
%load_ext autoreload
%autoreload 2
from src.models import *
from pytorch_lightning import Trainer

model = XRayResNet50()


# training
trainer = Trainer(max_epochs=1)
trainer.fit(model, train_loader)


[autoreload of src.models failed: Traceback (most recent call last):
  File "/geode2/home/u050/leckhoff/Carbonate/micromamba/envs/ml_base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/geode2/home/u050/leckhoff/Carbonate/micromamba/envs/ml_base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/geode2/home/u050/leckhoff/Carbonate/micromamba/envs/ml_base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/geode2/home/u050/leckhoff/Carbonate/micromamba/envs/ml_base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/geode2/home/u050/leckhoff/Carbonate/micromamba/envs/ml_base/lib/python3.11/site-packages/IPython/extensions/auto

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type               | Params
---------------------------------------------
0 | model | ResNet             | 23.5 M
1 | acc   | MulticlassAccuracy | 0     
---------------------------------------------
6.1 K     Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.057    Total estimated model params size (MB)
/geode2/home/u050/leckhoff/Carbonate/micromamba/envs/ml_base/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/geode2/home/u050/leckhoff/Carbonate/micromamba/envs/ml_base/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
